In [1]:
import tensorflow as tf
import numpy as np
import timeit
import time
from sklearn.utils import shuffle
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
from sklearn.utils import shuffle
from tensorflow.contrib.layers import flatten

<h1>Extract MNIST data</h1>

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",reshape=False,one_hot=True)
tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<h1>Prepare training, validation and testing data</h1>

In [4]:
x_train, y_train           = mnist.train.images, mnist.train.labels
x_validation, y_validation = mnist.validation.images, mnist.validation.labels
x_test, y_test             = mnist.test.images, mnist.test.labels

In [5]:
#pad images with 0s (28x28 to 32x32)
print(x_train.shape)
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_validation = np.pad(x_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
print(x_train.shape)
print(x_validation.shape)
print(x_test.shape)

(55000, 28, 28, 1)
(55000, 32, 32, 1)
(5000, 32, 32, 1)
(10000, 32, 32, 1)


In [6]:
x_train, y_train = shuffle(x_train, y_train)

<h1>Define hyperparameter</h1>

In [22]:
EPOCHS = 10
BATCH_SIZE = 32
lr = 0.001
tf.reset_default_graph()

<h1>Placeholder</h1>

In [23]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None, 10))

<h1>Define LeNet-5</h1>

In [24]:
def LeNet(x):    
    conv1_w = tf.Variable(tf.truncated_normal(shape = [5,5,1,6], mean = 0, stddev = 0.1))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x,conv1_w, strides = [1,1,1,1], padding = 'VALID') + conv1_b 
    conv1 = tf.nn.relu(conv1)

    pool_1 = tf.nn.max_pool(conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')

    conv2_w = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = 0, stddev = 0.1))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(pool_1, conv2_w, strides = [1,1,1,1], padding = 'VALID') + conv2_b

    conv2 = tf.nn.relu(conv2)
    pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID') 
    
    fc1 = flatten(pool_2)
 
    fc1_w = tf.Variable(tf.truncated_normal(shape = (400,120), mean = 0, stddev = 0.1))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1 = tf.matmul(fc1,fc1_w) + fc1_b
    
    fc1 = tf.nn.relu(fc1)

    fc2_w = tf.Variable(tf.truncated_normal(shape = (120,84), mean = 0, stddev = 0.1))
    fc2_b = tf.Variable(tf.zeros(84))
    fc2 = tf.matmul(fc1,fc2_w) + fc2_b
    fc2 = tf.nn.relu(fc2)
    
    fc3_w = tf.Variable(tf.truncated_normal(shape = (84,10), mean = 0 , stddev = 0.1))
    fc3_b = tf.Variable(tf.zeros(10))
    predicted_logits = tf.matmul(fc2, fc3_w) + fc3_b
    return predicted_logits

<h1>Cost and optimization</h1>

In [27]:
def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [25]:
predicted_logits = LeNet(x)
loss_operation = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = predicted_logits, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate = lr)
training_operation = optimizer.minimize(loss_operation)

In [26]:
correct_prediction = tf.equal(tf.argmax(predicted_logits, 1), tf.argmax(y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

<h1>Training, validating, testing</h1>
<h2>1. Print out validation accuracy after each training epoch</h2>
<h2>2. Print out training time on each epoch</h2>
<h2>3. Print out testing accuracy</h2>

In [29]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(x_train)
    for i in range(EPOCHS):
        start_time = time.time()
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = x_train[offset:end,:], y_train[offset:end,:]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        use_time = time.time() - start_time   
        validation_accuracy = sess.run(accuracy_operation,feed_dict={x:x_validation, y:y_validation})
        print("epoch {0}, Validation Accuracy :{2:.3f}".format(i+1, use_time, validation_accuracy))

    test_accuracy = sess.run(accuracy_operation,feed_dict={x:x_test, y:y_test})
    print("Test Accuracy:{:.3f}".format(test_accuracy))

epoch 1, Validation Accuracy :0.979
epoch 2, Validation Accuracy :0.981
epoch 3, Validation Accuracy :0.984
epoch 4, Validation Accuracy :0.986
epoch 5, Validation Accuracy :0.988
epoch 6, Validation Accuracy :0.988
epoch 7, Validation Accuracy :0.986
epoch 8, Validation Accuracy :0.985
epoch 9, Validation Accuracy :0.989
epoch 10, Validation Accuracy :0.989
Test Accuracy:0.989
